In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_selection import VarianceThreshold,mutual_info_classif,SelectKBest,SelectFromModel

import lightgbm as lgb

from sklearn.model_selection import  GridSearchCV,cross_val_score,TimeSeriesSplit,cross_validate
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from sklearn import tree
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, recall_score, precision_score, roc_auc_score,roc_curve, auc,matthews_corrcoef,f1_score,brier_score_loss,make_scorer, metrics


/Users/neenu/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#reading csv files
X_train=pd.read_csv('X_train_F.csv')
Y_train=pd.read_csv('Y_train_F.csv')
X_test=pd.read_csv('X_test_F.csv')
Y_test=pd.read_csv('Y_test_F.csv')

In [3]:
X_Data=pd.read_csv('X_Data.csv')
Y_Data=pd.read_csv('Y_Data.csv')

In [4]:
X_train= X_train[:70000]
Y_train=Y_train[:70000]

In [5]:
# with default paramter
clf = lgb.LGBMClassifier(class_weight="balanced")
feature_selector = SelectKBest(mutual_info_classif, k=15)
pipeline_lgbm  = Pipeline([('feature_sele',feature_selector),('clf',clf)])

In [6]:
pipeline_lgbm.fit(X_train,Y_train.values.ravel())
y_trainpred=pipeline_lgbm.predict_proba(X_train)
y_testpred = pipeline_lgbm.predict_proba(X_test)

In [7]:
# auc with default parameters
print('train auc with default parameter:',roc_auc_score(Y_train,y_trainpred[:,1]))
print('test auc with default parameter:',roc_auc_score(Y_test,y_testpred[:,1]))

train auc with default parameter: 0.8760980162408469
test auc with default parameter: 0.6993909779423524


In [8]:
print('brier_score_loss:',brier_score_loss(Y_test,y_testpred[:,1]))

brier_score_loss: 0.1539796447040106


In [9]:
y_testpred_ =pipeline_lgbm.predict(X_test)
print("Recall:\n",metrics.recall_score(Y_test,y_testpred_))
print("matthews_corrcoef:\n",matthews_corrcoef(Y_test, y_testpred_))

Recall:
 0.49582530507385997
matthews_corrcoef:
 0.15912901174213082


In [10]:
#  parameters currently used
print('Parameters currently in use:\n')
print(clf.get_params())

Parameters currently in use:

{'boosting_type': 'gbdt', 'class_weight': 'balanced', 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}


In [12]:
#GridSearchCV and feature selection in nested CV

time_splitter_inner = TimeSeriesSplit(n_splits = 3)
time_splitter_outer= TimeSeriesSplit(n_splits = 10)

model=lgb.LGBMClassifier(class_weight="balanced")
params = {"max_depth": [5,10,15],
              "learning_rate" : [0.1,0.15,0.3],
              "num_leaves": [32,150,200],
              "n_estimators": [500,700],
              'boost_from_average': [False],
               "random_state": [2]}
feature_selector = SelectKBest(mutual_info_classif, k=25)

gs_lgbm = GridSearchCV(model, param_grid = params, cv = time_splitter_inner, scoring = 'roc_auc',verbose=10, n_jobs=-1)

pipelinelgbm  = Pipeline([('feature_sele',feature_selector),
                      ('clf_cv',gs_lgbm)])

In [13]:
pipelinelgbm .fit(X_Data,Y_Data.values.ravel())

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  3.3min finished


Pipeline(steps=[('feature_sele',
                 SelectKBest(k=25,
                             score_func=<function mutual_info_classif at 0x7fde5bd08e60>)),
                ('clf_cv',
                 GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
                              estimator=LGBMClassifier(class_weight='balanced'),
                              n_jobs=-1,
                              param_grid={'boost_from_average': [False],
                                          'learning_rate': [0.1, 0.15, 0.3],
                                          'max_depth': [5, 10, 15],
                                          'n_estimators': [500, 700],
                                          'num_leaves': [32, 150, 200],
                                          'random_state': [2]},
                              scoring='roc_auc', verbose=10))])

In [14]:
custom_scorer = {
                 'recall': make_scorer(recall_score),
                 'Brier': make_scorer(brier_score_loss),
                 'Matthewscorrcoef': make_scorer(matthews_corrcoef),
                 'ROC_AUC': make_scorer(roc_auc_score)
                 }
lgbm_scores = cross_validate(pipelinelgbm,X_Data,Y_Data.values.ravel(),cv = time_splitter_outer, scoring = custom_scorer)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:   22.6s finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:   47.6s finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   41.6s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  2.6min finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  3.0min finished


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   58.0s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  2.9min finished


In [15]:
sorted(lgbm_scores.keys())

['fit_time',
 'score_time',
 'test_Brier',
 'test_Matthewscorrcoef',
 'test_ROC_AUC',
 'test_recall']

In [16]:
lgbm_time = lgbm_scores['fit_time']
lgbm_recall = lgbm_scores['test_recall']
lgbm_Brier = lgbm_scores['test_Brier']
lgbm_Matthewscorrcoef = lgbm_scores['test_Matthewscorrcoef']
lgbm_ROC_AUC = lgbm_scores['test_ROC_AUC']

In [17]:
print("=== fit time ===")
print(lgbm_time)
print("=== All recall Scores ===")
print(lgbm_recall)
print("=== All Brier Scores ===")
print(lgbm_Brier)
print("=== All Matthewscorrcoef Scores ===")
print(lgbm_Matthewscorrcoef)
print("=== All ROC_AUC Scores ===")
print(lgbm_ROC_AUC)

=== fit time ===
[ 36.936867    78.76261091 123.80430889 154.17805815 195.05736399
 233.39290214 271.20169973 307.30292487 349.64998889 367.31504798]
=== All recall Scores ===
[0.2        0.4587156  0.34334764 0.46268657 0.39153439 0.33576642
 0.38666667 0.5        0.46153846 0.63529412]
=== All Brier Scores ===
[0.0259901  0.02007701 0.02846535 0.06449395 0.0409791  0.04276678
 0.06064356 0.14040154 0.13105061 0.17134213]
=== All Matthewscorrcoef Scores ===
[0.38487556 0.58971118 0.43955496 0.27892804 0.31512406 0.22084375
 0.26414654 0.1418235  0.17595267 0.18523249]
=== All ROC_AUC Scores ===
[0.59907828 0.72737311 0.66783805 0.70581648 0.68284894 0.65246625
 0.67183482 0.68317456 0.67162868 0.73429026]


In [18]:
print("=== Mean recall Score ===")
print( lgbm_recall.mean())
print("=== Mean Brier Score ===")
print( lgbm_Brier.mean())
print("=== Mean Matthewscorrcoef  Score ===")
print( lgbm_Matthewscorrcoef .mean())
print("=== Mean ROC_AUC  Score ===")
print( lgbm_ROC_AUC .mean())
print("=== Mean fit_time ===")
print( lgbm_time .mean())

=== Mean recall Score ===
0.4175549863723676
=== Mean Brier Score ===
0.07262101210121012
=== Mean Matthewscorrcoef  Score ===
0.2996192755509842
=== Mean ROC_AUC  Score ===
0.6796349432982927
=== Mean fit_time ===
211.76017725467682


In [19]:
print("=== Minimum recall Score ===")
print( lgbm_recall.min())
print("=== Minimum Brier Score ===")
print(  lgbm_Brier.min())
print("=== Minimum Matthewscorrcoef  Score ===")
print(  lgbm_Matthewscorrcoef.min())
print("=== Minimum ROC_AUC  Score ===")
print(  lgbm_ROC_AUC.min())
print("=== Minimum RF_time  ===")
print(  lgbm_time.min())

=== Minimum recall Score ===
0.2
=== Minimum Brier Score ===
0.020077007700770078
=== Minimum Matthewscorrcoef  Score ===
0.141823500374587
=== Minimum ROC_AUC  Score ===
0.5990782756664776
=== Minimum RF_time  ===
36.936866998672485


In [20]:
print("=== Maximum recall Score ===")
print( lgbm_recall.max())
print("=== Minimum Brier Score ===")
print( lgbm_Brier.min())
print("=== Maximum Matthewscorrcoef  Score ===")
print( lgbm_Matthewscorrcoef .max())
print("=== Maximum ROC_AUC  Score ===")
print( lgbm_ROC_AUC .max())
print("=== Maximum fit_time ===")
print( lgbm_time .max())

=== Maximum recall Score ===
0.6352941176470588
=== Minimum Brier Score ===
0.020077007700770078
=== Maximum Matthewscorrcoef  Score ===
0.5897111805909775
=== Maximum ROC_AUC  Score ===
0.7342902579223748
=== Maximum fit_time ===
367.31504797935486
